In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch earthquake data
def fetch_earthquake_data(days=3650, min_magnitude=2.5, output_file='earthquake_data.csv'):
    """
    Fetch earthquake data from the USGS API and save it as a CSV file.
    
    Parameters:
    days (int): Number of past days to fetch data for (default: 10 years ~ 3650 days).
    min_magnitude (float): Minimum magnitude threshold (default: 2.5).
    output_file (str): Name of the CSV file to save data.
    """
    
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=days)
    
    url = (
        f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
        f"&starttime={start_time.strftime('%Y-%m-%d')}"
        f"&endtime={end_time.strftime('%Y-%m-%d')}"
        f"&minmagnitude={min_magnitude}"
        f"&limit=20000"  # Max limit to get more data
    )
    
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data from USGS API")
        return
    
    data = response.json().get("features", [])
    
    # Extract relevant information
    records = []
    for quake in data:
        props = quake['properties']
        coords = quake['geometry']['coordinates']
        
        records.append({
            "Time": datetime.utcfromtimestamp(props['time'] / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            "Magnitude": props.get('mag', 'N/A'),
            "Depth_km": coords[2] if coords else 'N/A',
            "Latitude": coords[1] if coords else 'N/A',
            "Longitude": coords[0] if coords else 'N/A',
            "Location": props.get('place', 'Unknown')
        })
    
    # Save to CSV
    df = pd.DataFrame(records)
    df.to_csv(output_file, index=False)
    print(f"✅ Earthquake data saved to {output_file} (Total records: {len(df)})")

# Run the function
fetch_earthquake_data()


Error fetching data from USGS API
